### MNIST: Classifying Handwritten Digits
This project uses Tensorflow and Keras to classify handwritten digits

In [2]:
## Import needed libraries & modules
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist

##### Load the Dataset

In [3]:
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()


##### Justify your preprocessing

##### Explore the Dataset

In [4]:
print(tf.shape(X_train))

tf.Tensor([60000    28    28], shape=(3,), dtype=int32)


2023-09-13 06:26:33.145741: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2023-09-13 06:26:33.145776: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2023-09-13 06:26:33.145786: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2023-09-13 06:26:33.146468: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-09-13 06:26:33.146837: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
## This cell contains a function for showing 5 images from a dataloader – DO NOT CHANGE THE CONTENTS! ##
def show5(img_loader):
    dataiter = iter(img_loader)

    batch = next(dataiter)
    labels = batch[1][0:5]
    images = batch[0][0:5]
    for i in range(5):
        print(int(labels[i].detach()))

        image = images[i].numpy()
        plt.imshow(image.T.squeeze().T)
        plt.show()

In [ ]:
# Explore data
show5(train_loader)

## Build your Neural Network
Using the layers in `torch.nn` (which has been imported as `nn`) and the `torch.nn.functional` module (imported as `F`), construct a neural network based on the parameters of the dataset.
Use any architecture you like.

*Note*: If you did not flatten your tensors in your transforms or as part of your preprocessing and you are using only `Linear` layers, make sure to use the `Flatten` layer in your network!

In [ ]:
# Define the class for your neural network
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.activation = F.relu
        self.layer1 = nn.Linear(28 * 28 * 1, 128)
        self.layer2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.activation(self.layer1(x))
        x = self.layer2(x)
        return x

Specify a loss function and an optimizer, and instantiate the model.

If you use a less common loss function, please note why you chose that loss function in a comment.

In [ ]:
# Instantiate the model
net = Net()

# Choose an optimizer
optimizer = optim.Adam(net.parameters(), lr=0.001)

# Choose a loss function
criterion = nn.CrossEntropyLoss()

## Running your Neural Network
Use whatever method you like to train your neural network, and ensure you record the average loss at each epoch.
Don't forget to use `torch.device()` and the `.to()` method for both your model and your data if you are using GPU!

If you want to print your loss **during** each epoch, you can use the `enumerate` function and print the loss after a set number of batches. 250 batches works well for most people!

In [ ]:
num_epochs = 10

# Establish a list for our history
train_loss_history = list()

for epoch in range(num_epochs):
    net.train()
    train_loss = 0.0
    train_correct = 0
    for i, data in enumerate(train_loader):
        # data is a list of [inputs, labels]
        inputs, labels = data

        # Pass to GPU if available.
        if torch.cuda.is_available():
            inputs, labels = inputs.cuda(), labels.cuda()

        # Zero out the gradients of the optimizer
        optimizer.zero_grad()

        # Get the outputs of your model and compute your loss
        outputs = net(inputs)
        loss = criterion(outputs, labels)

        # Compute the loss gradient using the backward method and have the optimizer take a step
        loss.backward()
        optimizer.step()

        # Compute the accuracy and print the accuracy and loss
        _, preds = torch.max(outputs.data, 1)
        train_correct += (preds == labels).sum().item()
        train_loss += loss.item()
    print(f'Epoch {epoch + 1} training accuracy: {train_correct/len(train_loader):.2f}% training loss: {train_loss/len(train_loader):.5f}')
    train_loss_history.append(train_loss/len(train_loader))

Plot the training loss (and validation loss/accuracy, if recorded).

In [ ]:
# Plot the training and validation loss history
plt.plot(train_loss_history, label="Training Loss")
plt.legend()
plt.show()

## Testing your model
Using the previously created `DataLoader` for the test set, compute the percentage of correct predictions using the highest probability prediction.

If your accuracy is over 90%, great work, but see if you can push a bit further!
If your accuracy is under 90%, you'll need to make improvements.
Go back and check your model architecture, loss function, and optimizer to make sure they're appropriate for an image classification task.

In [ ]:
val_correct = 0
net.eval()
for inputs, labels in test_loader:
  if torch.cuda.is_available():
    inputs, labels = inputs.cuda(), labels.cuda()

  outputs = net(inputs)
  loss = criterion(outputs, labels)

  _, preds = torch.max(outputs.data, 1)
  val_correct += (preds == labels).sum().item()
print(f'Percentage of Correct Predictions: {val_correct/len(test_loader):.2f}')

## Improving your model

Once your model is done training, try tweaking your hyperparameters and training again below to improve your accuracy on the test set!

The model above is the most accurate. I tried a number of variations on the layers, activation functions, step size, and optimizers and until I happened upon the above model my accuract was more like 8-9%

## Saving your model
Using `torch.save`, save your model for future loading.

In [ ]:
save_path = '/net.pt'
torch.save(net.state_dict(), save_path)